In [1]:
%matplotlib inline

# adding project dirs to path so code may be referenced from the notebook
import sys
sys.path.insert(0, '..')

In [3]:
!pip install hydrotools

  Using cached hydrotools-2.2.2-py3-none-any.whl (6.1 kB)
  Using cached hydrotools.events-1.1.5-py3-none-any.whl (7.7 kB)
  Using cached hydrotools.nwis_client-3.2.1-py3-none-any.whl (15 kB)
  Using cached hydrotools.nwm_client-5.0.3-py3-none-any.whl (967 kB)
  Using cached hydrotools.metrics-1.3.3-py3-none-any.whl (10 kB)
  Using cached hydrotools._restclient-3.0.5-py3-none-any.whl (18 kB)
  Using cached h5netcdf-1.1.0-py2.py3-none-any.whl (26 kB)
  Using cached hydrotools.caches-0.1.3-py3-none-any.whl (4.4 kB)
  Using cached tables-3.8.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.5 MB)
  Using cached aiohttp-3.7.4.post0-py3-none-any.whl
  Using cached aiohttp_client_cache-0.8.1-py3-none-any.whl (31 kB)
  Using cached python_forge-18.6.0-py35-none-any.whl (31 kB)
  Using cached async_timeout-3.0.1-py3-none-any.whl (8.2 kB)
  Using cached chardet-4.0.0-py2.py3-none-any.whl (178 kB)
  Using cached url_normalize-1.4.3-py2.py3-none-any.whl (6.8 kB)
  Using cached aioht

In [4]:
import os
import config
# import const
import utils
# from hydrotools.nwm_client import gcp as nwm
from datetime import datetime, timedelta
# import pandas as pd
from usgs_to_parquet import fetch_usgs

In [5]:
start = datetime(2022, 12, 18)
download_period = timedelta(days=1)
number_of_periods = 40

sites = utils.get_usgs_gages()

# Fetch USGS gage data in daily batches
for p in range(number_of_periods):

    # Setup start and end date for fetch
    start_dt = (start + download_period * p)
    end_dt = (start + download_period * (p + 1))
    start_dt_str = start_dt.strftime("%Y-%m-%d")
    end_dt_str = end_dt.strftime("%Y-%m-%d")

    observations_data = fetch_usgs(
        sites=sites["gage_id"],
        start_dt=start_dt_str,
        end_dt=end_dt_str
    )

    # Filter out data not on the hour
    observations_data.set_index("value_time", inplace=True)
    obs = observations_data[
        observations_data.index.hour.isin(range(0, 23)) 
        & (observations_data.index.minute == 0) 
        & (observations_data.index.second == 0)
    ]
    obs.reset_index(level=0, allow_duplicates=True, inplace=True)

    # Save as parquet file
    parquet_filepath = os.path.join(config.USGS_PARQUET, f"{start_dt.strftime('%Y%m%d')}.parquet")
    utils.make_parent_dir(parquet_filepath)
    obs.to_parquet(parquet_filepath)

/srv/conda/envs/notebook/lib/python3.10/site-packages/hydrotools/nwis_client/iv.py:283: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  dfs.loc[:, "value"] = pd.to_numeric(dfs["value"], downcast="float")
/srv/conda/envs/notebook/lib/python3.10/site-packages/hydrotools/nwis_client/iv.py:283: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  dfs.loc[:, "value"] = pd.to_numeric(dfs["value"], downcast="float")
/srv/conda/envs/notebook/lib/python3.10/site-packages/hydrotools/nwis_client/iv.py:283: FutureWarning: In a future version, `df.iloc[:, i] = newvals` wi